> Solve Kuramoto–Sivashinsky with PIML

<center>$\large \frac{\partial u}{\partial t}=-v\frac{\partial^4 u}{\partial x^4}-\frac{\partial^2 u}{\partial x^2}-u\frac{\partial u}{\partial x}$</center>

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
import progressbar
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib inline

In [2]:
data = pd.read_parquet('Data/KS_data.parquet')

Scaling the data or PCA won't work

In [3]:
scaler = StandardScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data = data.applymap(lambda n: (n + 1) / 2) # set all values bewteen 0 and 1

In [4]:
data.loc[:10, :10]  # just an example

,0,1,2,3,4,5,6,7,8,9,10
0,-0.478193,-0.423484,-0.361481,-0.294564,-0.224610,-0.153012,-0.080711,-0.008241,0.064236,0.136886,0.210136
1,-0.422947,-0.359123,-0.291522,-0.221948,-0.151654,-0.081407,-0.011565,0.057864,0.127159,0.196825,0.267534
2,-0.355469,-0.286757,-0.217152,-0.147817,-0.079377,-0.012018,0.054417,0.120348,0.186393,0.253301,0.321885
3,-0.280954,-0.210939,-0.142184,-0.075218,-0.010091,0.053510,0.116160,0.178607,0.241705,0.306349,0.373406
4,-0.204034,-0.135486,-0.069613,-0.006376,0.054664,0.114221,0.173183,0.232528,0.293251,0.356305,0.422529
5,-0.128438,-0.063269,-0.001522,0.057319,0.114079,0.169773,0.225505,0.282389,0.341478,0.403704,0.469814
6,-0.056850,0.003817,0.060877,0.115222,0.167962,0.220317,0.273522,0.328744,0.387020,0.449199,0.515869
7,0.009049,0.064755,0.117113,0.167289,0.216590,0.266360,0.317883,0.372313,0.430609,0.493473,0.561285
8,0.068433,0.119234,0.167275,0.213910,0.260565,0.308631,0.359380,0.413892,0.472988,0.537174,0.606572
9,0.121133,0.167462,0.211848,0.255763,0.300679,0.347975,0.398855,0.454267,0.514849,0.580858,0.652112


In [5]:
go.Figure(data =
    go.Contour(
        z=data.loc[:1000].to_numpy()
    )).write_image('images/problem_3_data_example.png')

![](images/problem_3_data_example.png)

In [6]:
def test_n_components_acc(n, data):
    pca = PCA(n_components=n)
    reduced_data = pca.fit_transform(data)
    aux = pca.inverse_transform(reduced_data)
    return mean_absolute_error(data, aux)

In [ ]:
pca_n_acc = []
for i in progressbar.progressbar(range(2, 50)):
    pca_n_acc.append(test_n_components_acc(i, data))

# I started with n_components up to 50 above, but results were bad even for 50, I just extended the test
# in order to save time, that's why the X of the chart looks like that

for i in progressbar.progressbar(range(50, 200, 10)):
    pca_n_acc.append(test_n_components_acc(i, data))

 41% (20 of 48) |##########              | Elapsed Time: 0:01:38 ETA:   0:02:27

In [ ]:
px.line(x=[*range(2, 50), *range(50, 200, 10)], y=pca_n_acc,
        title='Reduced data difference to real').write_image('images/PCA_components_test.png')

![](images/PCA_components_test.png)

I'll use 90 as the number of components, that's the first one where the difference to original on return is negligible

In [5]:
pca = PCA(n_components=90)

In [6]:
%%time
reduced_data = pca.fit_transform(data)

CPU times: total: 58.2 s
Wall time: 5.75 s


In [7]:
mean_absolute_error(data, pca.inverse_transform(reduced_data))

0.0006817918148951832

In [9]:
reduced_data = pd.DataFrame(reduced_data)

scale the reduced data

In [10]:
red_scaler = StandardScaler()
reduced_data = pd.DataFrame(red_scaler.fit_transform(reduced_data))
reduced_data = reduced_data.applymap(lambda n: (n + 1) / 2) # set all values bewteen 0 and 1

---

### LSTM

I want to try a LSTM, gonna start with a relatively tame one

reshape data

In [11]:
sample_size = 1000
X, y = [], []
for i in range(len(reduced_data) - (sample_size + 1)):
    X.append(reduced_data.loc[i:i + sample_size - 1].to_numpy())
    y.append(reduced_data.loc[i + sample_size].to_numpy())

In [12]:
len(X), len(y)

(198999, 198999)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [16]:
model = keras.Sequential()
model.add(layers.LSTM(128, input_shape=(1000, 90)))
model.add(layers.Dense(90))
model.compile()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               112128    
                                                                 
 dense_1 (Dense)             (None, 90)                11610     
                                                                 
Total params: 123,738
Trainable params: 123,738
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(
    X_train, y_train, validation_data=(X_test, y_test), batch_size=1, epochs=1
)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

So the model is eating my whole vram even with batch_size = 1, there's no way I'm running this. Will change my approach to DeepONets

---

### DeepONets

In [19]:
import deepxde as dde

DeepXDE backend not selected or invalid. Use tensorflow.compat.v1.
Using backend: tensorflow.compat.v1



Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Instructions for updating:
non-resource variables are not supported in the long term
Enable just-in-time compilation with XLA.




In [27]:
data = dde.data.TripleCartesianProd(X_train, np.array(y_train), X_test, y_test)

ValueError: The training dataset does not have the format of Cartesian product.

I have no idea what shape is the input expected to be in, [the docs are vague](https://deepxde.readthedocs.io/en/latest/modules/deepxde.data.html#deepxde.data.triple.TripleCartesianProd) and [the examples are no better](https://deepxde.readthedocs.io/en/latest/demos/operator/antiderivative_aligned.html#complete-code)

---

### OpInf

[Willcox-Research-Group OpInf looks good and has docs, gonna try it](https://willcox-research-group.github.io/rom-operator-inference-Python3/content/index.html)

In [11]:
reduced_data

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,1.060983,-0.249232,0.033320,-0.003673,0.717194,1.046063,0.610880,0.881125,-0.095760,1.208036,...,1.287039,0.835713,0.230009,0.408131,0.915408,0.979728,0.321034,-0.053301,0.046553,0.741576
1,1.055380,-0.241545,0.043931,-0.020404,0.787182,1.059425,0.642775,0.824348,-0.074474,1.230836,...,1.288494,0.949028,0.215377,0.419659,0.920888,0.938208,0.465689,-0.180783,-0.155095,0.738815
2,1.049613,-0.227252,0.060362,-0.036550,0.858226,1.069472,0.671495,0.763695,-0.057550,1.249672,...,1.297449,1.066072,0.201506,0.379826,0.869458,0.857093,0.613904,-0.254435,-0.353925,0.800326
3,1.043500,-0.206530,0.082012,-0.051347,0.929366,1.075283,0.696153,0.701332,-0.045229,1.264819,...,1.295591,1.155701,0.167765,0.301866,0.753864,0.760580,0.732424,-0.240733,-0.494130,0.924109
4,1.036902,-0.179899,0.108162,-0.063958,0.999676,1.076241,0.715926,0.639420,-0.037566,1.276671,...,1.260132,1.200473,0.100077,0.216509,0.589493,0.684383,0.785061,-0.133949,-0.528797,1.082124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.064257,0.065702,0.992203,0.557008,0.867911,0.322929,0.235827,1.436795,0.970282,1.104490,...,-0.346586,-0.566914,1.144641,0.126556,0.530678,0.739130,-0.623557,0.313669,-0.228914,0.821861
199996,0.087371,0.072969,1.004596,0.553764,0.878036,0.313668,0.224850,1.444529,0.967536,1.095734,...,-0.320804,-0.582517,1.124550,0.190273,0.459681,0.720689,-0.654188,0.369592,-0.232887,0.751271
199997,0.109518,0.079295,1.017653,0.551714,0.887140,0.306317,0.214181,1.453414,0.963708,1.087367,...,-0.283037,-0.588845,1.101614,0.253078,0.401736,0.695244,-0.664626,0.441134,-0.244352,0.681439
199998,0.130722,0.084619,1.031319,0.550924,0.895317,0.300732,0.204020,1.463568,0.958981,1.079431,...,-0.235373,-0.585681,1.077122,0.314132,0.357592,0.665391,-0.657032,0.527681,-0.263055,0.614511
